In [1]:
# Packages
from huggingface_hub import hf_hub_download
from datasets import load_dataset
import os 
os.chdir("C:\\Users\\WulfN\\.cache\\huggingface\\hub\\")

# consider other packages that may improve performance of downloading data, and distributing model across PC cores
# consider AWS / GCP / Azure VM if pc cannot handle training the model

# download model
#hf_hub_download(repo_id = 'prithivMLmods/Deep-Fake-Detector-Model', filename = 'config.json') # presumably the config file is needed first
# model.safetensors, # pytorch_model.bin


# First, test model on huggingface deepfake data, then try on other data source.

# Datasets can be found here: https://huggingface.co/datasets?modality=modality:image&sort=trending&search=deepfake

# Download 4 ~400mb parquet files (testing with first one)
# hf_hub_download(repo_id="Whab/deepfake", filename="data/train-00000-of-00004-51c797bb91ec4f77.parquet", repo_type="dataset")


In [2]:
### Load Data
dataset = load_dataset("Whab/deepfake") # split="train"

# Datasets with higher quality deep fakes is needed

# Preview dataset images

In [5]:
#import ipywidgets as widgets
from IPython.display import display
import ipyplot
#from PIL import Image
#import matplotlib.pyplot as plt

preview_num = 5

sample_images = dataset['train'][1:preview_num + 1]

# display_images = [
#     widgets.Image(value=sample_images['image'][i].tobytes(), format='png', width=256, height=256) 
#     for i in range(preview_num)
# ]

# display_images
# display(widgets.HBox(display_images))

## Display images vertically
# for i in range(preview_num):
#     display(sample_images['image'][i])

# display(display_images) 
# #widgets.HBox(display_images)
# # hbox = widgets.HBox(display_images)
# # display(hbox)
# #display_images

In [3]:
### Pipeline approach
from transformers import pipeline

pipe = pipeline("image-classification", model="prithivMLmods/Deep-Fake-Detector-Model")


### Load model directly:
# from transformers import AutoImageProcessor, AutoModelForImageClassification

# processor = AutoImageProcessor.from_pretrained("prithivMLmods/Deep-Fake-Detector-Model")
# model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Deep-Fake-Detector-Model")

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

c:\Users\WulfN\Python Projects\virtual_env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\WulfN\.cache\huggingface\hub\models--prithivMLmods--Deep-Fake-Detector-Model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


preprocessor_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
# predictions

df_length = 100

predictions = pd.DataFrame()
for i in range(df_length):
    image_score = pd.DataFrame(pipe(dataset['train'][:df_length+1]['image'][i]))
    predictions = pd.concat([predictions, pd.concat({i: image_score}, names=['image_num'])])

predictions
# AUC score

# dist'n of fake and real scores



label     score
image_num                  
0         0  Fake  0.976109
          1  Real  0.020260
1         0  Fake  0.978068
          1  Real  0.018317
2         0  Fake  0.977322
...           ...       ...
97        1  Real  0.016506
98        0  Fake  0.979183
          1  Real  0.017317
99        0  Fake  0.971088
          1  Real  0.021595

[200 rows x 2 columns]